## Data Loading and Transformation

In [1]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable
import pandas as pd
import math
import csv

hh_df = pd.read_csv('../../Data/SMTO_2015/SMTO_2015_Households.csv')
ps_df = pd.read_csv('../../Data/SMTO_2015/SMTO_2015_Respondents.csv')
campus_info = pd.read_csv('Campus_Info.csv')

# Load relevant columns
df = ps_df[['pscampusattend', 'personstatusgrad', 'personstatustime', 'psuniversityinvolvednumyears']]
df = df.join(hh_df[['HmTTS2006', 'hhlivingsituation']])
df = df.rename(columns={'HmTTS2006': 'HomeZone', 'pscampusattend': 'Campus', 'hhlivingsituation': 'Family', 
                       'personstatusgrad': 'Level', 'personstatustime': 'Status', 'psuniversityinvolvednumyears': 'Years'})
df = df.dropna() # Remove rows with missing data

# Convert Campus column to numerical column
df['Campus'] = df["Campus"].apply(lambda x: campus_info.index[campus_info['School'] == x].tolist()[0])

# Convert columns to numerical
df['HomeZone'] = pd.to_numeric(df['HomeZone'], downcast='signed')
df['Family'] = (df['Family'] == 'Live with family/parents')*1

# All campuses available to all students
df['Available'] = 1

# Dataframe with walk distances
df_path = pd.read_csv('../../../LoS/Walk_Distances.csv')
origins = list(set(list(df_path['Origin'])))
dists = list(df_path['Data'])

# Function for distance lookup
not_found = set()
def find_distance(origin, destination):
    try:
        i = origins.index(origin)
    except ValueError:
        not_found.add(origin)
        return 0
    try:
        j = origins.index(destination)
    except ValueError:
        not_found.add(destination)
        return 0
    return dists[i*2392 + j] / 1000

# List of campus' TTS zones from Joven's MOE data
campus_zones = list(campus_info['Zone'])
school_codes = list(campus_info['Code'])

# Load distances into dataframe
for i in range(len(campus_zones)):
    df[school_codes[i] + "_DIST"] = df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i]))
print("# of zones not found:", len(not_found))

# Load enrollment data
enrollment_df = pd.read_csv('../../Data/Enrolment/Joven_Enrollment.csv').set_index('School')
def get_log_enrollment(level, school):
    return math.log1p(enrollment_df.loc[school][level]) # Level: UG, Grad, Total

# of zones not found: 127


## Running Model

In [2]:
def print_results(results, i):
    print("___________Segment " + str(i) + "__________")
    #print("n:" + str(results.getGeneralStatistics()['Sample size'][0]), "\tR^2", results.getGeneralStatistics()['Rho-square for the init. model'][0])
    print(results.getEstimatedParameters()[['Value', 'p-value']])
    print()

    
def run_model(run_name, num_segments, row_to_segment, segment_to_level = None):
    global df
    global full_results
    df_in_func = df.copy()
    df_in_func['Segment'] = df_in_func.apply(row_to_segment, axis=1)
    df_in_func = df_in_func.drop(columns=['Level', 'Status', 'Years', 'HomeZone', 'Family'])
    
    for i in range(num_segments):
        temp_df = df_in_func.copy()
        database = db.Database("SMTO", temp_df)
        globals().update(database.variables)
        database.remove(SG_DIST == 0) # Remove unknown distances
        database.remove(Segment != i)

        ASCs, V, av = [], {}, {}
        if segment_to_level: # Enrollment
            B_ENROLL = Beta('B_ENROLL', 0, None, None, 0)
            level = segment_to_level(i)
            log_enroll = []
            for j in range(len(school_codes)):
                log_enroll.append(get_log_enrollment(level, school_codes[j]))
                ASCs.append(Beta('ASC_' + school_codes[j], log_enroll[j], None, None, 1))
        else:
            for j in range(len(school_codes)):
                ASCs.append(Beta('ASC_' + school_codes[j], 0, None, None, 0 if school_codes[j] != 'YG' else 1))
        

        B_DIST = Beta('B_DIST', 0, None, None, 0)
        
      
        for j in range(len(school_codes)):
            av[j] = Available
            if segment_to_level:
                V[j] = B_ENROLL * ASCs[j] + B_DIST * database.variables[school_codes[j] + "_DIST"]
            else:
                V[j] = ASCs[j] + B_DIST * database.variables[school_codes[j] + "_DIST"]
            

        logprob = models.loglogit(V, av, Campus)

        biogeme = bio.BIOGEME(database, logprob, numberOfThreads=1)
        biogeme.modelName = run_name + str(i)
        results = biogeme.estimate(saveIterations=False)
        print_results(results, i)
        
        full_results['Sample'].append(results.getGeneralStatistics()['Sample size'][0])
        full_results['Log_Lhood'].append(results.getGeneralStatistics()['Final log likelihood'][0])
        full_results['Akaike'].append(results.getGeneralStatistics()['Akaike Information Criterion'][0])
        full_results['Bayesian'].append(results.getGeneralStatistics()['Bayesian Information Criterion'][0])
        full_results['B_DIST'].append(results.getEstimatedParameters()['Value']['B_DIST'])
        full_results['B_DIST_P'].append(results.getEstimatedParameters()['p-value']['B_DIST'])
        
        if segment_to_level:
            full_results['B_ENROLL'].append(results.getEstimatedParameters()['Value']['B_ENROLL'])
            full_results['B_ENROLL_P'].append(results.getEstimatedParameters()['p-value']['B_ENROLL'])
            for j in range(len(school_codes)):
                full_results['ASC_' + school_codes[j]].append(log_enroll[j])
        else:
            full_results['B_ENROLL'].append(0)
            full_results['B_ENROLL_P'].append(0)
            for code in school_codes:
                full_results['ASC_' + code].append(results.getEstimatedParameters()['Value']['ASC_' + code] if code != 'YG' else 0)

## Segmentation Method (must start at 0)

In [3]:
def segments13(x):
    if x.Level == 'Other':
        return 0
    elif x.Level == 'UG':
        return (0 if x.Status == 'FT' else 2) + (not x.Family) + (1 if x.Years == 0 else 5)
    else:
        return 9 + (0 if x.Status == 'FT' else 2) + (not x.Family)
def segments13_to_level(segment):
    if segment == 0:
        return 'Total'
    elif segment < 9:
        return 'UG'
    else:
        return 'Grad'
    
    
def segments2(x):
    return (not x.Family)*1
def segments2_to_level(segment):
    return 'Total'


def segments7(x):
    if x.Level == 'Other':
        return 0
    elif x.Level == 'UG':
        return (not x.Family) + (1 if x.Years == 0 else 3)
    else:
        return 5 + (not x.Family)
def segments7_to_level(segment):
    if segment == 0:
        return 'Total'
    elif segment  < 5:
        return 'UG'
    else:
        return 'Grad'
    

def segments4(x):
    return (not x.Family) + (0 if (x.Level == 'UG') else 2)
def segments4_to_level(segment):
    if segment < 2:
        return 'Grad' # Includes other
    else:
        return 'UG'
    

def miller_segments(x):
    if x.Level == 'Other':
        return 0
    elif x.Level == 'UG':
        return 3 if x.Status == 'PT' else (not x.Family) + 1
    else:
        return 6 if x.Status == 'PT' else (4 + (not x.Family))
def miller_segments_to_level(segment):
    if segment == 0:
        return 'Total'
    elif segment < 4:
        return 'UG'
    else:
        return 'Grad'

## Main Code

In [4]:
#run_model("LC_Segment_", 13, segments13, segments13_to_level)
#run_model("LC_ASC_Segment_", 13, segments13)
#run_model("LC_Bool_Segment_", 2, segments2, segments2_to_level)
#run_model("LC_Bool_ASC_Segment_", 2, segments2)
#run_model("LC_TriLevel_Segment_", 7, segments7, segments7_to_level)
#run_model("LC_TriLevel_ASC_Segment_", 7, segments7)
#run_model("LC_BiLevel_Segment_", 4, segments4, segments4_to_level)
#run_model("LC_BiLevel_ASC_Segment_", 4, segments4)


result_headers = ['Sample', 'B_DIST', 'B_DIST_P', 'B_ENROLL', 'B_ENROLL_P']
for code in school_codes:
    result_headers.append('ASC_' + code)
result_headers += ['Log_Lhood', 'Akaike', 'Bayesian']
full_results = {}
for header in result_headers:
    full_results[header] = []

        
run_model("LC_Miller_ASC_Segment_", 7, miller_segments)
run_model("LC_Miller_Segment_", 7, miller_segments, miller_segments_to_level)

___________Segment 0__________
           Value       p-value
ASC_MI -0.255495  6.000047e-01
ASC_OC -1.799090  1.947648e-02
ASC_RY  2.629828  0.000000e+00
ASC_SC -0.288939  5.641055e-01
ASC_SG  1.774762  5.191874e-08
ASC_YK  1.267303  2.607649e-04
B_DIST -0.094884  3.663736e-14

___________Segment 1__________
           Value   p-value
ASC_MI  1.143743  0.000000
ASC_OC  0.182230  0.073288
ASC_RY  2.308600  0.000000
ASC_SC  1.455208  0.000000
ASC_SG  2.293066  0.000000
ASC_YK  2.157680  0.000000
B_DIST -0.066481  0.000000

___________Segment 2__________
           Value       p-value
ASC_MI  0.575328  3.574231e-04
ASC_OC -0.220532  1.004076e-01
ASC_RY  1.059997  0.000000e+00
ASC_SC  0.958442  3.817204e-10
ASC_SG  1.907680  0.000000e+00
ASC_YK  1.966317  0.000000e+00
B_DIST -0.172368  0.000000e+00

___________Segment 3__________
           Value       p-value
ASC_MI  0.807756  7.517093e-03
ASC_OC  0.638424  3.219134e-02
ASC_RY  1.876380  5.733192e-13
ASC_SC  1.244199  1.644636e-05
ASC_SG

In [5]:
with open('Run_Results.csv', mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    headers = ("Segment",) + tuple(range(7)) + tuple(range(7))
    writer.writerow(headers)
    for header in result_headers:
        writer.writerow([header] + full_results[header])